In [6]:
import cv2
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (25, 20)

In [17]:
def compute_skew(image):
    image = cv2.bitwise_not(image)
    height, width = image.shape
    
    # Remove filter.
    # edges = cv2.Canny(image, 150, 200, 3, 5)
    
    lines = cv2.HoughLinesP(image, 1, np.pi/180, 100, minLineLength=width / 2.0, maxLineGap=20)
    angle = 0.0
    if lines is None or len(lines) == 0:
        print("No hough lines!")
    else:
        nlines = lines.size

        # Reshape from (n_lines,1,4) to (n_lines,4).
        lines = lines.reshape(lines.shape[0], 4)

        # Remove [0] because of the new shape.
        # for x1, y1, x2, y2 in lines[0]:
        for x1, y1, x2, y2 in lines:
            angle += np.arctan2(y2 - y1, x2 - x1)

        # The function cv2.getRotationMatrix2D recieves as input the
        # angle in degrees, so I converted the return
        # https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#getrotationmatrix2d
        #return angle / nlines

        angle /= nlines
    return angle*180/np.pi

In [19]:
def deskew(image, angle):
    image = cv2.bitwise_not(image)
    non_zero_pixels = cv2.findNonZero(image)
    center, wh, theta = cv2.minAreaRect(non_zero_pixels)

    root_mat = cv2.getRotationMatrix2D(center, angle, 1)
    rows, cols = image.shape
    rotated = cv2.warpAffine(image, root_mat, (cols, rows), flags=cv2.INTER_CUBIC)
    rotated = cv2.getRectSubPix(rotated, (cols, rows), center)
    rotated = cv2.bitwise_not(rotated)

    return rotated

In [20]:
for filepath in sorted(glob("cut/*.*.[Jj][Pp][Gg]")):
    filename = os.path.basename(filepath)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    angle = compute_skew(img)
    rotated = deskew(img.copy(), angle)
    
    print("{} = {}".format(filename, angle))
    cv2.imwrite("rotated/" + filename, rotated)


01-02.0000000.jpg = 0.0
01-02.0000001.jpg = 0.0
01-02.0000002.jpg = 0.0
01-02.0000003.jpg = 0.0
01-02.0000004.jpg = 0.0
01-02.0000005.jpg = 0.07482241706822591
01-02.0000006.jpg = 0.0
01-02.0000007.jpg = 0.0
01-02.0000008.jpg = 0.0
01-02.0000009.jpg = 0.0
01-02.0000010.jpg = 0.19954657212221485
01-02.0000011.jpg = 0.0
01-02.0000012.jpg = 0.0
01-02.0000013.jpg = 0.0034111182711708643
01-02.0000014.jpg = 0.0
01-02.0000015.jpg = 0.0
01-02.0000016.jpg = 0.0
03-04.0000001.jpg = 0.0
03-04.0000002.jpg = 0.0
03-04.0000003.jpg = 0.0
03-04.0000004.jpg = 0.0
03-04.0000005.jpg = 0.24847524097248871
03-04.0000006.jpg = 0.0
03-04.0000007.jpg = 0.0
03-04.0000008.jpg = 0.0
03-04.0000009.jpg = 0.0
03-04.0000010.jpg = 0.0
03-04.0000011.jpg = 0.0
03-04.0000012.jpg = -0.25043223633551376
No hough lines!
03-04.0000013.jpg = 0.0
03-04.0000014.jpg = 0.0
03-04.0000015.jpg = 0.25007075571056725
03-04.0000016.jpg = 0.0
05-06.0000000.jpg = 0.0
05-06.0000001.jpg = 0.0
05-06.0000002.jpg = -0.2500489067913128
05-06